<a href="https://colab.research.google.com/github/neel26desai/llm-hacker-recreation/blob/main/fastai_llmhacker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install tiktoken

In [ ]:
#for llama and hugging face
!pip install transformers
!pip install torch
!pip install accelerate
!pip install bitsandbytes
!pip install auto-gptq
!pip install optimum
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

## Playing with Tokenizer and OpenAI API

## Tokenizer

In [ ]:
import tokenize, ast
from io import BytesIO
from tiktoken import encoding_for_model

Tokenizing -is representing a text interms of numbers, where the numbers are called the tokens. The process is also called encoding text.

"text-davinci-003" is a text model offered by OpenAI, we will be using the same tokenizer as the model, to tonkenize our text

In [ ]:
from tiktoken import encoding_for_model
#using the encoder usewd by the model text-davinci-003
enc = encoding_for_model('text-davinci-003')
#the string which we want to encode
txt = 'Today is quite cold init'
#encoding the text to tokens
tokens = enc.encode(txt)
tokens

[8888, 318, 2407, 4692, 2315]

In [ ]:
#we hav represented text as number using the encoder, now we can convert the text back tu numbers using the decoder
[enc.decode_single_token_bytes(i).decode('utf-8') for i in tokens]

['Today', ' is', ' quite', ' cold', ' init']

## OpenAI API

In [ ]:
from openai import ChatCompletion, Completion
import openai

In [ ]:
#setting up the open ai api key, the key is saved as a secret on collab
from google.colab import userdata
#creating an OpenAI Client, the way to use cchat completion shown in referencenotebook is not longer supported, so we have to use the client method
client = openai.OpenAI(api_key=userdata.get('openai_key'))

In [ ]:
#telling the system what it is
consultant_sys = 'You are am Abroad Studies Cosultant LLM that is is familiar with all the the requirements for applying too different universities present in USA along with all the visa requirements and restrictions'
#creating a chat completion object
c = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role":"system","content":consultant_sys},
        {"role":"user","content":"What is the average GRE score to get into SJSU Masters in Software Engineering Program,keep the range as narrow as possible"}
    ]
)

In [ ]:
#raw output of completion.create
c

ChatCompletion(id='chatcmpl-8sM1Vxl4pM5fuTvu9FrqIWylqecGP', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The average GRE score required for admission to the Master's in Software Engineering program at San Jose State University (SJSU) can vary each year. However, based on previous admissions data, a competitive range for the average GRE score for admission to the program is typically around 310-320. It's important to note that meeting the average GRE score does not guarantee admission, as the admissions committee considers various factors such as GPA, letters of recommendation, statement of purpose, and relevant work experience as well.", role='assistant', function_call=None, tool_calls=None))], created=1707965061, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=101, prompt_tokens=72, total_tokens=173))

In [ ]:
#formatted response
print(c.choices[0].message.content)

The average GRE score required for admission to the Master's in Software Engineering program at San Jose State University (SJSU) can vary each year. However, based on previous admissions data, a competitive range for the average GRE score for admission to the program is typically around 310-320. It's important to note that meeting the average GRE score does not guarantee admission, as the admissions committee considers various factors such as GPA, letters of recommendation, statement of purpose, and relevant work experience as well.


In [ ]:
# get information rearding the token used in the prompt response
c.usage.dict()

{'completion_tokens': 138, 'prompt_tokens': 72, 'total_tokens': 210}

In [ ]:
#computing the cost for the call based on the token number
print('$ Cost for GPT3.5 ', 0.002/1000*210)
print('$ Cost if GPT4  was used', 0.03/1000*210)

$ Cost for GPT3.5  0.00041999999999999996
$ Cost if GPT4  was used 0.006299999999999999


#### Follow Up in the same conversation

In follow up the entire conversation is sent back to the model

In [ ]:

#creating a chat completion object
c = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role":"system","content":consultant_sys},
        {"role":"user","content":"What is the average GRE score to get into SJSU Masters in Software Engineering Program,keep the range as narrow as possible"},
        {"role":"assistant","content":" The average GRE score required would be between 310-320 quantitative and verbal sections combined"},
        {"role":"user","content":"I have an GRE scoe of 310, with 3 years of work experience, woeking as an MLOps Engineer, based on this can you recommend some good University where I can get in for MS SE "}
    ]
)

In [ ]:
print(c.choices[0].message.content)

Based on your GRE score of 310 and 3 years of work experience as an MLOps Engineer, here are some universities that you can consider for pursuing a Master's in Software Engineering:

1. San Jose State University (SJSU)
2. North Carolina State University (NCSU)
3. University of Texas at Austin (UT Austin)
4. University of Southern California (USC)
5. Northeastern University
6. University of Illinois at Urbana-Champaign (UIUC)
7. University of California, Irvine (UCI)
8. University of Maryland, College Park (UMCP)
9. University of Florida (UF)
10. University of California, San Diego (UCSD)

Please note that admission decisions are based on various factors such as GPA, letters of recommendation, statement of purpose, and work experience in addition to your GRE score. It is always recommended to research each university's admission requirements and criteria before submitting an application.


In [ ]:
# creating function that will make it easy for us to ask GPT
def ask_gpt(client,user,system=None,model='gpt-3.5-turbo',**kwargs):
  '''
  This funtion can handle only one user text completion request
  client: open ai cient object
  user: consists the message enterned by the user that needs to be completed
  system: system configuration, telling gpt what it is
  model: open ai model which you want to use
  kwargs: extra arguments neeed

  the function returns the chat completion object
  '''
  msgs = []
  #if system argument exists add it to the list of messages else dont
  if system:
    msgs.append({"role":"system","content":system})
  #add the user's message to the list of messages
  msgs.append({"role":"user","content":user})
  #creating a client and chat completion object
  c = client.chat.completions.create(
      model = model,
      messages = msgs,
      **kwargs
  )
  #
  return c

In [ ]:
def print_response(c):
  print(c.choices[0].message.content)


In [ ]:
print_response(ask_gpt(client,'Which are the best universities for pursuing PhD in AI',system = consultant_sys))

There are several top universities in the United States that offer excellent programs for pursuing a PhD in Artificial Intelligence (AI). Here are some of the best universities known for their AI research and programs:

1. Massachusetts Institute of Technology (MIT): MIT is renowned for its cutting-edge research in AI, and its Computer Science and Artificial Intelligence Laboratory (CSAIL) is a leading center for AI studies.

2. Stanford University: Stanford's Artificial Intelligence Laboratory is highly regarded in the field of AI research and offers a variety of AI-related PhD programs.

3. Carnegie Mellon University: Carnegie Mellon's School of Computer Science is globally recognized for AI research, with its Robotics Institute being a prominent hub of AI innovation.

4. University of California, Berkeley (UC Berkeley): UC Berkeley's Department of Electrical Engineering and Computer Sciences is considered one of the top-tier institutions for AI research and offers various AI-focused

In [ ]:
#telling the system what it is
comedian_sys = 'You are an Comedian LLM that is really good at sarcasm and roasts'
#creating a chat completion object
c = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role":"system","content":comedian_sys},
        {"role":"user","content":"I really got dumped by my girlfriend on valentines day"}
    ],

)

In [ ]:
c

ChatCompletion(id='chatcmpl-8sO4s5Rw7rEenuyPA5PNhBwxOI6U4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Well, congratulations on achieving a special level of singledom. I mean, getting dumped on the most romantic day of the year? That's like winning an Olympic gold medal in heartbreak. But hey, at least you can now join the exclusive club of people who have experienced the bittersweet taste of love and rejection simultaneously. It's a true badge of honor, my friend. Who needs a significant other when you can have a good old, heart-wrenching breakup story, right?", role='assistant', function_call=None, tool_calls=None))], created=1707972958, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=99, prompt_tokens=40, total_tokens=139))

In [ ]:
c.choices[0].message.content

"Well, congratulations on achieving a special level of singledom. I mean, getting dumped on the most romantic day of the year? That's like winning an Olympic gold medal in heartbreak. But hey, at least you can now join the exclusive club of people who have experienced the bittersweet taste of love and rejection simultaneously. It's a true badge of honor, my friend. Who needs a significant other when you can have a good old, heart-wrenching breakup story, right?"

## Creating our own Code interpreter

In [ ]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [ ]:
#the function description, is more important than the function itself
def multiply(a:int, b:int=1):
    "Multiply a * b"
    return a * b


In [ ]:
#the below function is taken from https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb
def schema(f):
  '''
  This function is used to convert a python function in to a json schema, which we will pass in our call
  '''
  kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
        for n,o in inspect.signature(f).parameters.items()}
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()
  return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [ ]:
schema(multiply)

{'name': 'multiply',
 'description': 'Multiply a * b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `multiply`',
  'type': 'object'}}

In [ ]:
#prompting gpt and passing our function schema. We also specify to gpt that it is only allowed to use the function we ar passing anf nothing else
c=ask_gpt(client,"Use the `multiply` function to solve this: What is 1024 into 2?",
           system = "You must use the `multiply` function instead of adding yourself.",
           functions=[schema(multiply)])

In [ ]:
c

ChatCompletion(id='chatcmpl-8sOhWO9TFYq8kaLP78gAsnK7NUe4Z', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "a": 1024,\n  "b": 2\n}', name='multiply'), tool_calls=None))], created=1707975354, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=22, prompt_tokens=85, total_tokens=107))

In [ ]:
#checking out the return statement
c.choices[0].message.function_call

FunctionCall(arguments='{\n  "a": 1024,\n  "b": 2\n}', name='multiply')

In [ ]:
# checking the arguments given to the function
args = c.choices[0].message.function_call.arguments
print(args)

{
  "a": 1024,
  "b": 2
}


In [ ]:
#specify the allowed function calls, we are going to be write the python function below
funcs_ok = {'multiply', 'run_python_code'}
def call_func(c):
    #extracting the function from our response
    fc = c.choices[0].message.function_call
    #if the funcation call is not allowed return
    if fc.name not in funcs_ok:
      return print(f'Not allowed: {fc.name}')
    #if function called is allowed, call the function by passing it the arguments
    f = globals()[fc.name]
    #return the output of the function call
    return f(**json.loads(fc.arguments))

In [ ]:
call_func(c)

2048

In [ ]:
#the below function is taken from https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb
import ast  # Import the abstract syntax tree module

def run_python_code(code):
  '''
  this function executes a given piece of Python code and returns the result of the last expression in that code
  '''
  # Parse the input code string into an abstract syntax tree (AST)
  tree = ast.parse(code)

  # Get the last node in the body of the AST, or None if the body is empty
  last_node = tree.body[-1] if tree.body else None

  # If the last node is an expression, modify the AST to capture the result
  if isinstance(last_node, ast.Expr):
      # Create a target for the assignment (a variable named '_result')
      tgts = [ast.Name(id='_result', ctx=ast.Store())]
      # Create an assignment node that assigns the value of the expression to '_result'
      assign = ast.Assign(targets=tgts, value=last_node.value)
      # Replace the last node in the body with the assignment node
      tree.body[-1] = ast.fix_missing_locations(assign)

  # Create a namespace dictionary to execute the code in
  ns = {}
  # Compile the AST into executable code and execute it in the namespace
  exec(compile(tree, filename='<ast>', mode='exec'), ns)
  # Return the value of '_result' from the namespace, or None if it doesn't exist
  return ns.get('_result', None)


In [ ]:
run_python_code("""
a=9
b=5
a*b
""")

45

In [ ]:
#the below function is taken from https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb
def run_python_code(code: str):
    """
    This function allows the execution of arbitrary Python code,  the code to this function will be provided from the response of GPT
    Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`

    """
    # Prompt the user to confirm whether to proceed with execution
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')

    # Check if the user's response is not 'y' (case-insensitive)
    if go.lower() != 'y':
        # If the user does not confirm, return '#FAIL#'
        return '#FAIL#'

    # If the user confirms, execute the code using the `run` function and return the result
    return run(code)


In [ ]:
schema(run_python_code)

{'name': 'run_python_code',
 'description': '\n    This function allows the execution of arbitrary Python code,  the code to this function will be provided from the response of GPT\n    Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`\n\n    ',
 'parameters': {'properties': {'code': {'title': 'Code', 'type': 'string'}},
  'required': ['code'],
  'title': 'Input for `run_python_code`',
  'type': 'object'}}

In [ ]:
#asking GPT  to perform integration
c = ask_gpt(client,"What is integral of 12x when x ranges from 1 to 0?",
           system = "Use python for any required computations.",
           functions=[schema(run_python_code)])

In [ ]:
# passing the received completion obejct to call_func, which will pass the code to the python and run function for execution
ans = call_func(c)

Proceed with execution?
```
import sympy as sp

x = sp.Symbol('x')
f = 12*x
integral = sp.integrate(f, (x, 0, 1))
integral.evalf()
```
y


We got the answer that we wanted. Now lets say instead of it just throwing the answer at it we want it to tell us the answer in a proper fashion.
In that case in the chat completion the first prompt will be from the user role and the second prompt will be from the function role

In [ ]:
c = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    functions = [schema(run_python_code)],
    messages=[
        {"role":"user","content":"What is integral of 12x when x ranges from 1 to 0?"},
        #second role will be function, and for its content put the asnwer you got above
        {"role":"function","name":"run_python_code","content":str(ans)}
    ]

)


In [ ]:
print_response(c)

The integral of 12x from 1 to 0 is equal to 6.


In [ ]:
# when we ask it to use oython for any computation, and the question which we ask doesn;t involve any code in that case it will just ignore the function and give us a normal response
c = ask_gpt(client,"What is the longest bone in the human body",
            system="use python for any computation",
            functions=[schema(run_python_code)])


In [ ]:
print_response(c)

The longest bone in the human body is the femur, which is located in the thigh.


## Pytorch and Huggingface

In [ ]:
!pip install bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch
from google.colab import userdata
import time

### Deafult llama-2, with int parameters

In [ ]:
try:
    del model
    del res
except:
  pass
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#we will using the llama 2 with 7 billion parameter in hugging face
model_name = "meta-llama/Llama-2-7b-hf"

In [ ]:

#getting the model from hugging face
#load_in_8bit when set to true will set all the model parameter to use 8bit float rather than 16 bit, device map 0 is for GPU
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=0, load_in_8bit=True, use_auth_token = userdata.get('colab_hg_read'))

#### Tokenizing

In [ ]:
#get the modesl tokenizer
tokr = AutoTokenizer.from_pretrained(model_name,use_auth_token = userdata.get('colab_hg_read'))
#llama 2 is a simple llm and can only do text completion
prompt = 'Every night in my dreams I see you, I' #this is the song from titanic
#tokenizing the promp, return_tensors='pt' makes the function return pytorch tensor
tokens = tokr(prompt,return_tensors='pt')

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
tokens

{'input_ids': tensor([[    1,  7569,  4646,   297,   590, 12561, 29879,   306,  1074,   366,
         29892,   306]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

input_ids: numerical representations of your tokens

attention_masks: binary masks, which tells if the token should be included or not

In [ ]:
#just to confirm that the tokesn are correct, we will try to detokenize them and see if we get the same text back
tokr.batch_decode(tokens['input_ids'])

['<s> Every night in my dreams I see you, I']

Now we will call the generate function fo the model, it will redcursivelly call the model again again passing the the text it generated previously to generate new text.

the max_token argument of generate, determines how many new tokens are to be generated.

.to('cuda') will make sure that the model runs on the GPU.

After generation we will return the result to CPU so we can display it

In [ ]:
%%timeit
#unpacking the tokens dictionary and passing it to generate
res  = model.generate(**tokens.to("cuda"),max_new_tokens = 25).to("cpu")

5.32 s ± 656 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#the output of the model generate will tokens, which we will have to decode
print(res)
#decoding the output
tokr.batch_decode(res)

tensor([[    1,  7569,  4646,   297,   590, 12561, 29879,   306,  1074,   366,
         29892,   306,  4459,   366, 29889,   306,  6159,   363,   366, 29892,
           306,   817,   366, 29889,   306,   864,   366, 29892,   306,  5360,
           366, 29889,   306, 29915, 29885, 10771,   373]])


["<s> Every night in my dreams I see you, I feel you. I reach for you, I need you. I want you, I love you. I'm stuck on"]

### Llama2 with parameters saved as torch.bfloat16

We got tha above result when we used model with all parameters as int.  However we use there is another format called pytorch.bfloat16 that is useable on any morden GPU. However it will take twice as much RAM, but will it should give us a significant improvement in terms of time taken for prediction

In [ ]:
#loading model with parameters being of type bfloat16
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=0, torch_dtype=torch.bfloat16,use_auth_token = userdata.get('colab_hg_read'))

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
%%timeit
res = model.generate(**tokens.to("cuda"), max_new_tokens=25).to('cpu')


1.28 s ± 198 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#decoding the output
tokr.batch_decode(res)

["<s> Every night in my dreams I see you, I feel you that is how I know you'll always be close to me, Cause every night in my dreams I"]

We acan see using bfloat16, got the running time down from 5.32 second to 1.28 seconds.

### Illama 2 - GPTQ

In [ ]:
torch.cuda.empty_cache()

Using the GTPQ version which is an even more optimized version then the bfloat16. It is fine tunes on lower dimension data, and internammly uses a 4 bit representation of parameters and internally casts it to 16bit

In [ ]:
model = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ', device_map=0, torch_dtype=torch.float16,use_auth_token = userdata.get('colab_hg_read'))

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
%%timeit
#unpacking the tokens dictionary and passing it to generate
res  = model.generate(**tokens.to("cuda"),max_new_tokens = 25).to("cpu")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


1.09 s ± 187 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print(res)
print(tokr.batch_decode(res))

tensor([[    1,  7569,  4646,   297,   590, 12561, 29879,   306,  1074,   366,
         29892,   306,  4459,   366,   393,   338,   920,   306,  1073,   366,
         29915,   645,  2337,   367,  3802,   304,   592, 29892,   315,  1071,
          1432,  4646,   297,   590, 12561, 29879,   306]])
["<s> Every night in my dreams I see you, I feel you that is how I know you'll always be close to me, Cause every night in my dreams I"]


We can see that the GPTQ version with torch.float16 was even quicker than the torch.bfloat16. This model is optimized to an extend that we can even load the 13 billion parameter version of the model on the standard GPU on collab

#### 13 billion paraameter version

In [ ]:
#free Up Sapce
try:
  del model,res
except:
  pass
torch.cuda.empty_cache()

In [ ]:
#getting the 1 billion parameter version of the model
mn = 'TheBloke/Llama-2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16,use_auth_token = userdata.get('colab_hg_read'))

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
%%time
res = model.generate(**tokens.to("cuda"), max_new_tokens=25).to('cpu')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


CPU times: user 1.88 s, sys: 12.7 ms, total: 1.89 s
Wall time: 2.03 s


We were able to run the entire model in **2 seconds**

In [ ]:
print(res)
print(tokr.batch_decode(res))

tensor([[    1,  7569,  4646,   297,   590, 12561, 29879,   306,  1074,   366,
         29892,   306,  4459,   366, 29889,    13, 29902,  5193,   304,   366,
         29892,   306,  6232,   411,   366,   590,  6483,   342, 22183,  1372,
           322,   590,  6501,   342,  8866, 29879, 29889]])
['<s> Every night in my dreams I see you, I feel you.\nI talk to you, I share with you my deepest secrets and my darkest fears.']


In [ ]:
#writing a function that helps to put everything together
def gen(prompt,model,maxlen=15,sample=True):
  #tokenizing the prompt
  toks = tokr(prompt, return_tensors="pt")
  #generating the next words
  res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
  #returning the decoded version of the text generation
  return tokr.batch_decode(res)

In [ ]:
#just to remember what the prompt
print(prompt)

Every night in my dreams I see you, I


In [ ]:
gen(prompt,model,50)

["<s> Every night in my dreams I see you, I feel you. That is how I know you go on, and on... and on.\nEvery night in my dreams you're alive, I feel you, I hear you, I know you. That is how I know that you go"]

In [ ]:
#free Up Sapce
try:
  del model,res
except:
  pass
torch.cuda.empty_cache()

### Instruction tuned LLM StableBeluga with 7 billion parameters

Each Instruction tuned llm has a different prompt format, you can view the beluga's prompt format over here https://huggingface.co/stabilityai/StableBeluga-7B

Beluga is built on top of llama 2 by fine tuning it

In [ ]:
#getting the model
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16,use_auth_token = userdata.get('colab_hg_read'))

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well and can complete songs. Help as much as you can.\n\n"

In [ ]:
def make_prompt(user, syst=sb_sys):
  #takes the user and system prompt and returns a string that in the prompt format supported by belugga
  return f"{syst}### User: {user}\n\n### Assistant:\n"

In [ ]:
prompt="Every night in my dreams I see you, what comes next in this song"
prompt=make_prompt(prompt)
print(prompt)

### System:
You are Stable Beluga, an AI that follows instructions extremely well and can complete songs. Help as much as you can.

### User: Every night in my dreams I see you, what comes next in this song

### Assistant:



In [ ]:
#tokenize
toks = tokr(prompt, return_tensors="pt")

In [ ]:
##generate words
res =  model.generate(**toks.to("cuda"), max_new_tokens=200, do_sample=True).to('cpu')

In [ ]:
res.shape

torch.Size([1, 262])

In [ ]:
tokr.batch_decode(res)

['<s> ### System:\nYou are Stable Beluga, an AI that follows instructions extremely well and can complete songs. Help as much as you can.\n\n### User: Every night in my dreams I see you, what comes next in this song\n\n### Assistant:\n Here\'s the complete song from which the lyrics come:\n\nEvery night in my dreams I see you,  \nCrying in the rain,  \nYou look like an angel,  \nBut you\'re acting like you\'ve never done this before,  \nIs it love that you\'re feeling or is it just a cold, cold heart?  \nBut I\'ll be your shoulder to cry on,  \nI\'ll be your lover when you\'re cold,  \nAnd when the last tear\'s been wiped away,  \nI\'ll still be on your side,  \nI will always be beside you,  \nI will stand by you.\n\nThe next lyrics might be "Just like a lover should, I\'ll always be there for you", or "Just like a soldier, I\'ll always stand beside you". They depend on which version of the song you']

### Llama 2 fine tuned on OpenOrca-Platypus2 with 13 billion parameters GPTQ version

GPTQ - meaning it is going to be quantized

Thr prompt format for this model can be found here https://huggingface.co/Open-Orca/OpenOrca-Platypus2-13B

In [ ]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/900 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [ ]:
def mk_openorca_prompt(user):
  #formatting the user prompt, to the format required by openorca
  return f"### Instruction: {user}\n\n### Response:\n"

In [ ]:
prompt="Every night in my dreams I see you, what comes next in this song"
prompt = mk_openorca_prompt(prompt)
print(prompt)

### Instruction: Every night in my dreams I see you, what comes next in this song

### Response:



In [ ]:
#getting the llama2 tokenizer
tokr = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf',use_auth_token = userdata.get('colab_hg_read'))

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
#using the model fo
#tokenize the prompt
toks = tokr(prompt, return_tensors="pt")
##generate words
res =  model.generate(**toks.to("cuda"), max_new_tokens=200, do_sample=True).to('cpu')
##decoding the generated tokens
tokr.batch_decode(res)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


['<s> ### Instruction: Every night in my dreams I see you, what comes next in this song\n\n### Response:\nI see you in my dreams every night,\nAnd with each passing day,\nI feel you slipping away,\nAn ocean apart but always in my heart.\n\nAs the melody rises, the chorus begins, repeating the phrase "We’re worlds apart" while expressing the emotional longing and distance between the singers.\n\nIn these dreams, I\'m falling so far from you,\nNever knowing if I\'ll see you again,\nYour touch is like heaven, let\'s stay this way,\nAs the skies change, we may grow apart,\n\nThe second verse describes the character\'s longing for the beloved while emphasizing the contrast of heavenly touch and growing apart.\n\nTonight, we\'ll share this sky,\nIf only for a moment,\nBefore we\'re separated by the sky,\nAnd we fly so far out of reach,\n\n']

There is still some hellucination, as I am expecting the model to complete the song sung by Celine Dion

### Retrival Augmented Generation

What happens here is we take the question that has been asked, and hen we try and search for documents that could help use answer the question.  We ca retrive the information from wikipedia or in our case use a songs api to get the lyrics.

We will tell the model to use the lyrics which we found to answer the question

##### Getting the Lyrics

In [ ]:
#Code generated by ChatGPT
import requests

def get_lyrics(artist, title):
    url = f"https://api.lyrics.ovh/v1/{artist}/{title}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        s= data.get("lyrics", "Lyrics not found")
        s=s.split('\n')
        return '\n'.join(s[1:])

    else:
        return "Failed to get lyrics"

# Example usage
artist = "Celine Dion"
title = "My Heart will go on"
lyrics = get_lyrics(artist, title)
print(lyrics)


Every night in my dreams
I see you, I feel you
That is how I know you go on
Far across the distance
And spaces between us
You have come to show you go on

Near, far, wherever you are
I believe that the heart does go on
Once more, you open the door
And you're here in my heart
And my heart will go on and on

Love can touch us one time
And last for a lifetime
And never let go 'til we're gone

Love was when I loved you
One true time I'd hold to
In my life, we'll always go on

Near, far, wherever you are
I believe that the heart does go on
(Why does the heart go on?)
Once more, you open the door
And you're here in my heart
And my heart will go on and on

You're here, there's nothing I fear
And I know that my heart will go on
We'll stay forever this way
You are safe in my heart
And my heart will go on and on

Mm, mm-mm


In [ ]:
#from the aboove lyrics lets drop the f

In [ ]:
ques = 'Every night in my dreams I see you, what comes next in this song'
question_with_context = f"""Answer the question with the help of the provided context.

## Context

{lyrics}

## Question

{ques}"""


In [ ]:
#using the model fo
#tokenize the question_with_context
toks = tokr(question_with_context, return_tensors="pt")
##generate words
res =  model.generate(**toks.to("cuda"), max_new_tokens=250, do_sample=True).to('cpu')
##decoding the generated tokens
tokr.batch_decode(res)

['<s> Answer the question with the help of the provided context.\n\n## Context\n\nEvery night in my dreams\nI see you, I feel you\nThat is how I know you go on\nFar across the distance\nAnd spaces between us\nYou have come to show you go on\n\nNear, far, wherever you are\nI believe that the heart does go on\nOnce more, you open the door\nAnd you\'re here in my heart\nAnd my heart will go on and on\n\nLove can touch us one time\nAnd last for a lifetime\nAnd never let go \'til we\'re gone\n\nLove was when I loved you\nOne true time I\'d hold to\nIn my life, we\'ll always go on\n\nNear, far, wherever you are\nI believe that the heart does go on\n(Why does the heart go on?)\nOnce more, you open the door\nAnd you\'re here in my heart\nAnd my heart will go on and on\n\nYou\'re here, there\'s nothing I fear\nAnd I know that my heart will go on\nWe\'ll stay forever this way\nYou are safe in my heart\nAnd my heart will go on and on\n\nMm, mm-mm\n\n## Question\n\nEvery night in my dreams I see y

In [ ]:
s=tokr.batch_decode(res)[0].split('##')

In [ ]:
s[2]

' Question\n\nEvery night in my dreams I see you, what comes next in this song?\n\nThe provided context does not include the full lyrics of the song. However, the next lines of the song with the given context are:\n\n"That is how I know you go on\nFar across the distance"\n\nFollowed by:\n\n"And spaces between us\nYou have come to show you go on"</s>'

We can see that the model, completed the lyrics correctly

Now the question is how do we know which page of what lyrics infomration is going to be the most useful.

For this we can use a special type of model, which can helpus find the similairty betweenthe prompt and the lyrics/article we chose to provide for contextx.

#### Finding the Right Context
We will use the BAAI/bge-small-en-v1.5 and passit out lyrics and the prompts, it will then generate aan embeddings forthem, after that can find the disimilairities between their embeddings

In [ ]:
#with out this the installation of sentence transformer is giving error
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=0)

In [ ]:
#gettig lyrics of a secod some for test, letsgo with Coldplay's fix you
fix_you_lyrics = get_lyrics('Coldplay', 'Fix You')
fix_you_lyrics

"[Chris Martin]\nWhen you try your best, but you don't succeed\nWhen you get what you want, but not what you need\nWhen you feel so tired, but you can't sleep\nStuck in reverse\nAnd the tears come streaming down your face\nWhen you lose something you can't replace\nWhen you love someone, but it goes to waste\nCould it be worse?\n\n[Chris & Will]\nLights will guide you home\nAnd ignite your bones\nAnd I will try to fix you\n\n[Chris Martin]\n\nAnd high up above or down below\nWhen you're too in love to let it go\nBut if you never try, you'll never know\nJust what you're worth\n\n[Chris & Will]\nLights will guide you home\nAnd ignite your bones\nAnd I will try to fix you\n\n[Breakdown: Chris, Jonny, Guy, Will]\nTears stream down your face\nWhen you lose something you cannot replace\nTears stream down your face, and I\nTears stream down your face\nI promise you I will learn from my mistakes\nTears stream down your face, and I\n\n\n[Chris Martin]\nLights will guide you home\nAnd ignite you

In [ ]:
ques_emb,my_hear_emb,fix_you_emb = emb_model.encode([question_with_context,lyrics,fix_you_lyrics], convert_to_tensor=True)


In [ ]:
print(ques_emb.shape,my_hear_emb.shape,fix_you_emb.shape)

torch.Size([384]) torch.Size([384]) torch.Size([384])


The for each text, the model generated an embedding of size 384

In [ ]:
#lets find the similairty between question and my heart will go on lyrics
import torch.nn.functional as F
print('Cosine similarity between the question and My Heart Will go on lyrics',F.cosine_similarity(ques_emb,my_hear_emb,dim=0))
print('Cosine similarity between the question and Fix You lyrics',F.cosine_similarity(ques_emb,fix_you_emb,dim=0))

Cosine similarity between the question and My Heart Will go on lyrics tensor(0.8966, device='cuda:0')
Cosine similarity between the question and Fix You lyrics tensor(0.7211, device='cuda:0')


In [ ]:
F.cosine_similarity(ques_emb,my_hear_emb,dim=0).to('cpu')

tensor(0.8966)

We can see thatthe similairty between my prtompt and lyrics of My heart will go on is higher, than of one that with the lyrics of fix you.

So when we have hundreds of documents from which we have to decide which would be the correct document to provide contex, n that case we can use this embedding method.  However when we have millions of documents in that case we have to use something called as a vector database, where we can save the encvodings of all our documents.

# Llama CCP

In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.44-cp310-cp310-manylinux_2_35_x86_64.whl size=2591553 sha256=e983e52aed9c9acf97766b83a09c9412dd0140e689c9c2f542d186e5ae0e26bf
  Stored in directory: /root/.cache/pip/wheels/6e/f0/52/1716aa7fefc7eb2a9b76775b0a61fc131b7dcc961e310a048a
Successfully built llama-cpp-python


In [ ]:
from llama_cpp import Llama

In [ ]:
llm = Llama(model_path="/content/drive/MyDrive/llama-2-7b-chat.Q4_K_M.gguf")

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /content/drive/MyDrive/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_co

In [ ]:
output = llm("Q: Which is the closest galaxy to Milky Way? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)


llama_print_timings:        load time =    7523.83 ms
llama_print_timings:      sample time =      17.23 ms /    32 runs   (    0.54 ms per token,  1857.33 tokens per second)
llama_print_timings: prompt eval time =    7523.73 ms /    17 tokens (  442.57 ms per token,     2.26 tokens per second)
llama_print_timings:        eval time =   18731.82 ms /    31 runs   (  604.25 ms per token,     1.65 tokens per second)
llama_print_timings:       total time =   26380.27 ms /    48 tokens


In [ ]:
print(output['choices'])

[{'text': 'Q: Which is the closest galaxy to Milky Way? A:  The closest galaxy to the Milky Way is the Canis Major Dwarf Galaxy, which is located about 25,000 light', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]


# Fine Tunning

These are just attempts for successful implementation refer to https://github.com/neel26desai/llm-hacker-recreation/blob/main/finetuning_with_axoltol.ipynb

### Using axolotl

In [ ]:
!pip3 install "axolotl[flash-attn,deepspeed] @ git+https://github.com/OpenAccess-AI-Collective/axolotl"

  Cloning https://github.com/OpenAccess-AI-Collective/axolotl to /tmp/pip-install-gsa37oov/axolotl_cd11001c188c420a9777f654116f2ec2
  Running command git clone --filter=blob:none --quiet https://github.com/OpenAccess-AI-Collective/axolotl /tmp/pip-install-gsa37oov/axolotl_cd11001c188c420a9777f654116f2ec2
  Resolved https://github.com/OpenAccess-AI-Collective/axolotl to commit 5a5d47458d9aaf7ead798d15291ba3d9bef785c5
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-install-gsa37oov/peft_c29e47d4f517419db1b88533377780aa
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-install-gsa37oov/peft_c29e47d4f517419db1b88533377780aa
  Resolved https://github.com/huggingface/peft.git to commit 7b7e4b2194344b3cd9edf4541eb13b083617f00d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.c

In [ ]:
torch.__version__

'2.1.0+cu121'

Various dependecies require different versions of pytorch, cause of which we are unable to run install it

### Using a Trainer Object from transformer object

We will be fine tning the model stabilityai/StableBeluga-7B on knowrohit07/ArithmeLogicm dataset

In [ ]:
dataset = 'knowrohit07/ArithmeLogic'
#using the stabilityai/StableBeluga-7B
model = AutoModelForCausalLM.from_pretrained('stabilityai/StableBeluga-7B', device_map=0,torch_dtype=torch.bfloat16)
#getting the llama2 tokenizer
tokr = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf',use_auth_token = userdata.get('colab_hg_read'))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
import datasets
#getting the dataset
arithematic_logic = datasets.load_dataset(dataset)

In [ ]:
arithematic_logic

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 28200
    })
})

In [ ]:
arithematic_logic['train'][0]

{'instruction': 'Rohit is saving money for a new wallet which costs $100. Rohit has only half of the money he needs. His parents decided to give him $15 for that purpose, and his grandparents twice as much as his parents. How much more money does Rohit need to buy the wallet?',
 'output': "Rohit has only 100 * 0.5 = $<<100*0.5=50>>50.\nRohit's grandparents gave him 15 * 2 = $<<15*2=30>>30.\nIn total, Rohit needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.\n#### 5"}

In [ ]:
arithematic_logic['train'][0]['instruction']

'Rohit is saving money for a new wallet which costs $100. Rohit has only half of the money he needs. His parents decided to give him $15 for that purpose, and his grandparents twice as much as his parents. How much more money does Rohit need to buy the wallet?'

The dataset has only instruction and output columns, it has 28200 records. For the sake of experimentation, we will train and test on the same split

In [ ]:
def convert_to_prompt(prompt):


  prompt_template=f'''### Instruction:

  {prompt}

  ### Response:

  '''

  return prompt_template

def tokenize_txt(prompt):
  return tokr(prompt, return_tensors="pt")

def decode_response(prompt):
  return tokr.batch_decode(res)



In [ ]:
tokr.pad_token = tokr.eos_token
def preprocess(examples):
    inputs = [ convert_to_prompt(doc) for doc in examples["instruction"]]
    model_inputs = tokr(inputs,padding=True, truncation=True, max_length=356)
    labels = tokr(text_target=examples["output"],padding=True, truncation=True, max_length=256)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
#before we perform retrinining lets use see how the defualt model performs on the task
prompt = convert_to_prompt(arithematic_logic['train'][0]['instruction'])
toks = tokr(prompt,return_tensors="pt")
##generate words
res =  model.generate(**toks.to("cuda"), max_new_tokens=200, do_sample=True)
##decoding the generated tokens
tokr.batch_decode(res)

["<s> ### Instruction:\n\n  Rohit is saving money for a new wallet which costs $100. Rohit has only half of the money he needs. His parents decided to give him $15 for that purpose, and his grandparents twice as much as his parents. How much more money does Rohit need to buy the wallet?\n\n  ### Response:\n\n   To calculate the difference, we'll first find out the total amount Rohit needs: $100 - $15 = $85. \n\n   Now, we'll find out how much each of the three groups gives: \n   Parents: 15\n   Grandparents: 2 * 15 = 30\n   Total: 85 + 30 + 15 = 130\n\n   We found that Rohit needs $130 more money to buy the wallet.</s>"]

In [ ]:

#tokenizing the dataset
tokenized_logic = arithematic_logic.map(preprocess, batched=True)

Map:   0%|          | 0/28200 [00:00<?, ? examples/s]

In [ ]:
tokenized_logic['train'][0]

Retraining

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokr, padding=True)

In [ ]:
from transformers import Trainer, TrainingArguments
# Define training arguments

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/fine_tuned_beluga/'+"my_beluga/results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_logic['train'],
    data_collator=data_collator
)



In [ ]:
# Fine-tune the adapter
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 7.06 MiB is free. Process 200039 has 14.74 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 46.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# References:
1. https://github.com/openai/openai-python
2. https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb